# Item23: Accept Functions for Simple Interfaces Instead of Classes 

### 1.Make a breif review of passing functions as parameters

In [54]:
names = ['Socrates', 'Archimedes', 'Plato', 'Aristotle']
names.sort(key=lambda x: len(x))
print(names)

['Plato', 'Socrates', 'Aristotle', 'Archimedes']


### We pass a function as a parameter here, because 

### -> function is easier to describe and simpler to define than a class

###   

# <<< Before we start to read item23, let's review DEFAULTDICT >>>

###    
###     

### 2. Make a brief review of defaultdict(Item 46)

-> 文字通り、デフォルトの値を設定できる辞書型。

-> KeyErrrorの発生しない世界を作るwww

公式ドキュメント：
https://docs.python.jp/3/library/collections.html#collections.defaultdict

In [55]:
somedict = {}
print(somedict[3]) # KeyError

KeyError: 3

In [56]:
from collections import defaultdict

someddict = defaultdict(int)
print(someddict[3]) # print int(), thus 0

0


# <<< OK, We can start to check the example of Item 23 >>>

###    
# 項目２３：単純なインタフェースではクラスの代わりに関数を受けよう


## 他の言語だとフック(Hook)を実現するには抽象クラスを定義する必要があるけど、
## Pythonなら函数を定義すればよい

### 1. Demo: 先のDefaultdictをカスタマイズしたいなら

In [57]:
def log_missing():
    print('Key defautly added')
    return 0

In [58]:
current = {'green':12, 'blue':3}
to_be_added = [
    ('red',5),
    ('blue',20),
    ('orange',9)
]

result = defaultdict(log_missing,current)
print('Before',dict(result))

Before {'green': 12, 'blue': 3}


In [59]:
for key, amount in to_be_added:
    result[key] += amount

print('After',dict(result))

Key defautly added
Key defautly added
After {'green': 12, 'blue': 23, 'red': 5, 'orange': 9}


### 2. Demo: Keysがないの数量（count the total number of keys that were missing）

#### Make a brief review of defaultdict(Item 15)
#### クロージャ：関数内関数をクロージャとして機能させ、動的に関数を生成することも可能。    

In [60]:
def todays_weather(arg1):
    def return_weather():
        return 'It’s ' +  arg1 + ' today.'
    return return_weather

In [61]:
def added_with_report(current, to_be_added):
    added_count = 0
    
    def missing():
        nonlocal added_count # Stateful Closure
        added_count += 1
        return 0
    result = defaultdict(missing, current)
    for key, amount in to_be_added:
        result[key] += amount
    
    return result, added_count

In [62]:
day1 = todays_weather('sunny')
day2 = todays_weather('cloudy')

In [63]:
day1()

'It’s sunny today.'

In [64]:
day2()

'It’s cloudy today.'

In [65]:
# Define a helper function that uses such a closure as the default value hook

In [66]:
current = {'green':12, 'blue':3}
to_be_added = [
    ('red',5),
    ('blue',20),
    ('orange',9)
]

result, count = added_with_report(current,to_be_added)

In [67]:
print('After',dict(result))

After {'green': 12, 'blue': 23, 'red': 5, 'orange': 9}


In [68]:
print('Keys to be add count:',count)

Keys to be add count: 2


### 但し、クロージャ(Closure)を使えば、ステートフル(stateful)な函数も定義できて
### フックとして使えるけれども、読みづらくなる

#### 3. Demo: Keysがないの数量（count the total number of keys that were missing）

In [69]:
class CountMissing(object):
    def __init__(self):
        self.added = 0
    def missing(self):
        self.added += 1
        return 0

In [70]:
counter = CountMissing()
result = defaultdict(counter.missing, current)

In [71]:
current = {'green':12, 'blue':3}
to_be_added = [
    ('red',5),
    ('blue',20),
    ('orange',9)
]

for key,amount in to_be_added:
    result[key] += amount

In [72]:
print('Keys to be add count:',counter.added)

Keys to be add count: 2


### 但し、「CountMissingのClassはなんのために作った？」、「誰がmissing methodを呼びますか」、
### いろんな問題を抱えていまて、下のDefaultdictを使わないと、分かりづらくなる

In [75]:
class BetterCountMissing(object):
    def __init__(self):
        self.added = 0
    def __call__(self):
        self.added += 1
        return 0

In [83]:
current = {'green':12, 'blue':3}
to_be_added = [
    ('red',5),
    ('blue',20),
    ('orange',9)
]


counter = BetterCountMissing()
result = defaultdict(counter, current)
for key, amount in to_be_added:
    result[key] += amount

In [84]:
print('Keys to be add count:',counter.added)

Keys to be add count: 2


###  _ _call_ _ 

### インスタンス生成では__init__しか呼び出されない。しかし、一度生成されたインスタンスを関数っぽく引数を与えて呼び出せば、__call__が呼び出されるという仕組み

###    
# まとめ

### 1. Instead of defining and instantiating classes, functions are often all you need for simple interfaces between components in Python.
(単純なインタフェースではクラスの代わりに関数を受けよう)

### 2. References to functions and methods in Python are first class, meaning they can be used in expressions like any other type.
（Pythonでパラメータ値として別の関数に渡すできる）

### 3. The __call__ special method enables instances of a class to be called like plain Python functions.
(クラスが__call__あったら、関数っぽく呼び出しことができる）

### 4. When you need a function to maintain state, consider defining a class that provides the __call__ method instead of defining a stateful closure (see Item 15: “Know How Closures Interact with Variable Scope”).
(状態を保守するために関数が必要な場合、状態を持つクロージャを定義する代わりに、__call__ メソッドを提供するクラスを定義することを考える。)